In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import datetime
import time

In [2]:
chrome_path = r'dependency/chromedriver'
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)

In [3]:
driver.get("https://oracleselixir.com/player/BrTT/championPool")

now = datetime.datetime.now()
date_csv = "{:02d}/{:02d}/{}".format(now.day,now.month,now.year)

wait = WebDriverWait(driver, 5)
start_date_input = wait.until(ec.presence_of_element_located(
        (By.XPATH, '//*[@id="start_date"]')), message='Campo "Start Date" era esperado e não foi encontrado.')
start_date_input.send_keys('01/01/2021')

time.sleep(3)

end_date_input = driver.find_element_by_xpath('//*[@id="end_date"]')
end_date_input.send_keys(date_csv)

time.sleep(3)

games_played_element = driver.find_elements_by_xpath("//*[contains(text(), 'Sivir')]/following::div/following::div")[0]
win_rate_element = driver.find_elements_by_xpath("//*[contains(text(), 'Sivir')]/following::div/following::div/following::div")[0]
kda_element = driver.find_elements_by_xpath("//*[contains(text(), 'Sivir')]/following::div/following::div/following::div/following::div/following::div")[0]

print('GP: ' + games_played_element.text)
print('WR: ' + win_rate_element.text)
print('KDA :' + kda_element.text)

driver.quit()

GP:1
WR:0%
KDA:8.0
